In [6]:
# Imports
from sdlabs import strategy,world
from sdlabs.utility import *

First, a "campaign" or list of campaigns will be created. A campaign consists of an instance of SDL.utility.CampaignInfo. It is a vessel for carrying all information necessary to perform a set of (V)SDL runs. It begines with building an environment. To do so, declare the relevant experiments instances from SDL.world. Check that module for the current options. Then, decide which environment to build. Current option is only VSDLEnvironment. Then, the ML agent will ne to be declared. Decide which to use from SDL.strategy, then feed it the necessary arguments (if deviating from defaults). Finally, you are ready to put it all together into a CampaignInfo instance. You can adjust some campaign arguments, like the number of runs to perform, the sampling procedure, the campaign name, etc.

In [7]:
# Create the experiments
experiments = {
    'BladeCoat': world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': world.Stability(cost=10.0),
}

# Create the environment
environment = world.VSDLEnvironment(experiments=experiments)

# Create the agent
agent = strategy.ArchitectureOne(epsilon=0.3)

# Combine into a campagin
campaign1 = CampaignInfo(
    name = 'Architecture Zero -- Environment 000',
    runs = 2,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 3),
        ('number_of_batches', 2),
        ('samples_per_batch', 2)
        ],
)

# Create another campaign
experiments = {
    'BladeCoat': world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': world.Stability(cost=10.0),
}
environment = world.VSDLEnvironment(experiments=experiments)
agent = strategy.ArchitectureOne(epsilon=0.7)
campaign2 = CampaignInfo(
    name = 'Architecture Zero -- Environment 001',
    runs = 2,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 2),
        ('number_of_batches', 3),
        ('samples_per_batch', 1)
        ],
)

# Compile into a campaign list
campaign_list = [campaign1,campaign2]

Next, the campaigns are run. If the number of campaigns and the number of samples to be performed are small, they can be run right in this notebook. If there are many samples, many runs, and/or many campaigns, consider running them using HPC. To do this, dump the campaign list into a binary file using pickle. Then, use the runSDL.py module in this repo to read and run the binary file.

In [9]:
# Run campaign(s) in notebook
for campaign in campaign_list:
    campaign.run()
    data = read_output(
        campaign.name + '.out.txt',
        data=SDLOutputData(
                campaign.name, 
                calc_stability=campaign.environment.experiments['Stability'].calc_stability
        )
    )
    campaign.dump_to_MAE()
    for experiment_name in [k for k,v in campaign.environment.experiments.items() if v.category == 'processing']:
        reference_inputs = campaign.environment.experiments[experiment_name].get_input_space()[1]
        reference_targets = data.calc_stability(reference_inputs)
        MAE = {run: get_cumulative_MAE(data,run,reference_inputs,reference_targets) for run in range(1,data.runs+1)}
        campaign.dump_to_MAE(MAE)


# Dump to a binary input file, to be run on an HPC
dump_campaign_list(campaign_list, 'my_VSDL_inputs - {} - .pkl'.format(datetime.datetime.now()))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

Next, read the output file (and optionally the MAE file) generated from a campaign. The data will be read in as an instance of SDL.utility.SDLOutputData.

In [ ]:
campaign = campaign_list[0]
MAEfilename = campaign.name + '.MAEout.txt'
MAE = 

In [5]:
# Read in output from a campaign
data = read_output('Architecture Zero -- Environment 000.out.txt')
for k,v in data.__dict__.items():
    print(k,v)

name Architecture Zero -- Environment 000
color #
calc_stability None
runs 2
sampling_procedure [('Initial Datapoints per Processing Variable', 3), ('Number of Batches', 2), ('Samples per Batch', 2)]
states {1: [[[<sdlabs.material.State object at 0x7f8eb83d9e80>, <sdlabs.material.State object at 0x7f8eb83d9670>], [<sdlabs.material.State object at 0x7f8eb83d9490>, <sdlabs.material.State object at 0x7f8ee8373fd0>], [<sdlabs.material.State object at 0x7f8ee8373ee0>, <sdlabs.material.State object at 0x7f8eb8394820>], [<sdlabs.material.State object at 0x7f8eb84c34c0>, <sdlabs.material.State object at 0x7f8eb84c3970>], [<sdlabs.material.State object at 0x7f8eb84c3bb0>, <sdlabs.material.State object at 0x7f8eb84c37f0>], [<sdlabs.material.State object at 0x7f8eb84c3730>, <sdlabs.material.State object at 0x7f8eb84c3c70>], [<sdlabs.material.State object at 0x7f8eb84c3040>, <sdlabs.material.State object at 0x7f8eb84c3280>], [<sdlabs.material.State object at 0x7f8eb84c3a30>, <sdlabs.material.State